In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url_bases = ['https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Demographics&Cycle=', 
             'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Dietary&Cycle=',
             'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Examination&Cycle=',
             'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Laboratory&Cycle=',
             'https://wwwn.cdc.gov/nchs/nhanes/search/variablelist.aspx?Component=Questionnaire&Cycle=',
             ]
years = ['1999-2000', '2001-2002', '2003-2004', '2005-2006', '2007-2008', '2009-2010', '2011-2012', '2013-2014', '2015-2016', '2017-2018']

In [5]:
for year in years:
    df = pd.DataFrame(columns=['var_name', 'file_name'])
    for url_base in url_bases:
        url = url_base + year
        request = requests.get(url)
        soup = BeautifulSoup(request.text, 'html.parser')
        
        for tag in soup.find_all('tr'):
            row = []
            for inner_tag in tag.find_all('td'):
                row.append(inner_tag.text.strip())
            if len(row) > 0:
                df = df.append({'var_name': row[0], 'file_name': row[2] + '.XPT'}, ignore_index=True)
        
    df.to_csv('data/' + year + '.csv', index=False)

In [6]:
def make_var_file_dict(data):
    var_file_dict = {}
    for index, row in data.iterrows():
        var_file_dict[row['var_name']] = row['file_name']
    return var_file_dict

In [10]:
with open("Output.txt", "w") as text_file:
    for year in years:
        year_data = pd.read_csv('data/' + year + '.csv')
        var_file_map = make_var_file_dict(year_data)
        print(f"VAR_TO_FILENAME_{year} = {var_file_map}", file=text_file)
    print("\n\n", file=text_file)